In [0]:
from delta.tables import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime, timedelta, date
from pyspark.sql.window import Window
import time
import os
import pandas as pd


In [0]:
%sql
SELECT BATCH_CD, count(*) FROM mypatronage_test111
GROUP BY ALL

In [0]:
%sql
SELECT BATCH_CD, count(*) FROM mypatronage_test111
WHERE BATCH_CD = 'CG' AND RecordStatus IS TRUE and Applicant_Type = 'Primary Caregiver'
GROUP BY ALL

In [0]:
containerPath = '/mnt/vac20sdpasa201vba/ci-vba-edw-2/DeltaTables/'
mostRecentRequestPath = None
for entity in dbutils.fs.ls(containerPath):
  print(entity.name)
  print(entity)

In [0]:
df = spark.read.format('delta').load(containerPath + 'DW_ADHOC_RECURR.DOD_PATRONAGE_SCD_PT/')
df.createOrReplaceTempView('DOD_PATRONAGE_SCD_PT') # name of the Delta Table you are reading

In [0]:
df = spark.read.format("delta").load("/mnt/ci-vba-edw-2/DeltaTables/DW_ADHOC_RECURR.DOD_PATRONAGE_SCD_PT/")

In [0]:
%sql
SELECT * FROM DELTA.`/mnt/ci-vba-edw-2/DeltaTables/DW_ADHOC_RECURR.DOD_PATRONAGE_SCD_PT/`
-- `/mnt/vac20sdpasa201vba/ci-vba-edw-2/DeltaTables/DW_ADHOC_RECURR.DOD_PATRONAGE_SCD_PT/`
                    

In [0]:
%sql
SELECT PTCPNT_VET_ID, count(1)
FROM delta.`/mnt/vac20sdpasa201vba/ci-vba-edw-2/DeltaTables/DW_ADHOC_RECURR.DOD_PATRONAGE_SCD_PT`
GROUP BY ALL
HAVING count(1) > 1

In [0]:
%sql
SELECT count(DISTINCT ICN, EDIPI) FROM mypatronage_test111
-- 6386058


In [0]:
cg_unix_start_time = int(time.mktime(datetime(2024, 12, 18, 23, 59, 59).timetuple())) * 1000
print(cg_unix_start_time)
others_unix_start_time = int(time.mktime(datetime(2023, 11, 1, 0, 0, 0).timetuple())) * 1000
print(others_unix_start_time)
date_today = datetime.today().date()
print(date_today) #date only
date_timestamp = datetime.today()
print(date_timestamp) #timestamp
print(date_timestamp.strftime("%A, %d %B %Y [Month Number: %m]" )) #formatting
print(int(time.mktime(date_today.timetuple())))
# print(int(time.mktime(cg_unix_start_time.timetuple()))) #1698796800000



In [0]:
%sql
with DateSeries as (
SELECT EXPLODE(SEQUENCE( DATE('2025-01-01'), last_day('2025-12-01'), INTERVAL 1 DAY )) as Year_2025
)
SELECT DATE_PART('DOW', Year_2025) FROM DateSeries
-- DATE_PART(EXPLODE(SEQUENCE( DATE('2025-01-01'), last_day('2025-12-01'), INTERVAL 1 DAY )), 'DAYOFWEEK') as Year_2025


In [0]:
%sql
WITH DateSeries AS (
SELECT SEQUENCE (
  DATE('2025-01-01'),
  last_day('2025-12-01'),
  INTERVAL 1 DAY
) AS DateArray
)
SELECT EXPLODE(DateArray) AS Date_Col FROM DateSeries
-- SELECT * FROM variant_explode()

In [0]:
%sql
SELECT EXPLODE(sequence(1, 10, 1)) as nbrs

In [0]:
%sql
SELECT DISTINCT SDP_Event_Created_Timestamp FROM mypatronage_test where BATCH_CD = 'SCD' ORDER BY 1 desc LIMIT 1 OFFSET 1 --(OFFSET starts with 0)

In [0]:
%sql
SELECT
  Batch_CD,
  COALESCE(Monday, 0) AS Monday,
  COALESCE(Tuesday, 0) AS Tuesday,
  COALESCE(Wednesday, 0) AS Wednesday,
  COALESCE(Thursday, 0) AS Thursday,
  COALESCE(Friday, 0) AS Friday,
  COALESCE(Saturday, 0) AS Saturday,
  COALESCE(Sunday, 0) AS Sunday
FROM
  (
    SELECT
      Batch_CD,
      date_format(SDP_Event_Created_Timestamp, 'EEEE') as Weekday,
      count(*) as RecordCounts
    FROM
      mypatronage_test -- WHERE extract(year from SDP_Event_Created_Timestamp) >= extract(YEAR FROM CURRENT_DATE) --filters records based on the current year
    GROUP BY
      ALL
  ) PIVOT (
    ROUND(
      sum(RecordCounts) / weekofyear(current_date()) -1,
      0
    ) FOR Weekday IN (
      "Monday" as Monday,
      "Tuesday" as Tuesday,
      "Wednesday" as Wednesday,
      "Thursday" as Thursday,
      "Friday" as Friday,
      "Saturday" as Saturday,
      "Sunday" as Sunday
    )
  )
GROUP BY ALL 
-- WHERE Batch_CD = "CG"

In [0]:
%sql
SELECT 
   ICN, SC_Combined_Disability_Percentage as Disability_Percent, 
   PT_Indicator, filename, RecordLastUpdated, SDP_Event_Created_Timestamp as InsertedOn, 
   RecordStatus, 
   count(ICN) OVER (PARTITION BY ICN ) as cnt 
FROM mypatronage_test
QUALIFY cnt >=2

In [0]:
%sql
SELECT ICN, PT_Indicator, SC_Combined_Disability_Percentage, filename from mypatronage where ICN in (select ICN
from pivoted_df
where `pivoted_df`.`June-24` is distinct from `pivoted_df`.`May-24` and (`pivoted_df`.`June-24` is not null and `pivoted_df`.`May-24` is not null))

In [0]:
%sql
select * from  read_files('/mnt/ci-vadir-shared/CPIDODIEX_20241005_spool.csv');


In [0]:
%sql
SELECT * FROM cloud_files_state(  "dbfs:/FileStore/streaming/scd/checkpoint/write_scd" )  -- path to checkpoint

In [0]:
%sql
SELECT DISTINCT input_file_name(), _metadata.file_modification_time,* FROM read_files(
    '/mnt/ci-vadir-shared/*.csv',
    format => 'csv', inferSchema=>True, modifiedAfter => to_date("2024-08-01 00:00:00", 'yyyy-MM-dd HH:mm:ss'))
where _metadata.file_modification_time > '2024-08-01' 

In [0]:
scd_schema1 = StructType([StructField('PTCPNT_ID', StringType()), 
                          StructField('CMBNED_DEGREE_DSBLTY', StringType()), 
                          StructField('DSBL_DTR_DT', StringType())]
)

In [0]:
new_cg_schema =  'Discharge_Revocation_Date__c STRING, Caregiver_Status__c STRING, CreatedById STRING, Dispositioned_Date__c STRING, CARMA_Case_ID__c STRING, Applicant_Type__c STRING, CreatedDate TIMESTAMP, Veteran_ICN__c STRING, Benefits_End_Date__c STRING, Caregiver_Id__c STRING, CARMA_Case_Number__c STRING, Caregiver_ICN__c STRING'


old_cg_schema = 'Discharge_Revocation_Date__c STRING, Caregiver_Status__c STRING, CreatedById STRING, Dispositioned_Date__c STRING, Applicant_Type__c STRING, CreatedDate TIMESTAMP, Veteran_ICN__c STRING, Benefits_End_Date__c STRING, Caregiver_Id__c STRING, Caregiver_ICN__c STRING'

In [0]:
%sql
select *, _metadata.file_modification_time, _metadata.file_path, rank() over(partition by Caregiver_ICN__c, Veteran_ICN__c, Applicant_Type__c order by  CreatedDate desc  ) rnk
from read_files('{cg_dir}', inferSchema=>False, header=>True,  schema=> '{use_schema}', modifiedAfter=> '{cg_max_date}') qualify rnk=1

In [0]:
cg_max_date = spark.sql("""SELECT cast(coalesce(max(from_unixtime(unix_timestamp(cast(SDP_Event_Created_Timestamp AS TIMESTAMP)))), '2023-10-30') as TIMESTAMP) cg_max_date 
            FROM mypatronage_test""").collect()[0][0]
cg_dir = '/mnt/ci-carma/landing/*.csv'
print(cg_max_date)
cg_max_date = '2023-10-30 00:00:00'

if datetime.strptime(cg_max_date, '%Y-%m-%d %H:%M:%S') < datetime.strptime('2024-08-08 00:00:00', '%Y-%m-%d %H:%M:%S'):
    use_schema = old_cg_schema
else:
    use_schema = new_cg_schema

cg_query = f"""
select *, _metadata.file_modification_time, _metadata.file_path, rank() over(partition by Caregiver_ICN__c, Veteran_ICN__c, Applicant_Type__c order by  CreatedDate desc  ) rnk
from read_files('{cg_dir}', inferSchema=>False, header=>True,  schema=> '{use_schema}', modifiedAfter=> '{cg_max_date}') qualify rnk=1"""

cg_raw_df = spark.sql(cg_query)
cg_raw_df.display()


In [0]:
%sql
SELECT
  version,
  operation,
  cast(operationMetrics.numTargetRowsInserted as INT) as Inserted,
  cast(operationMetrics.numTargetRowsUpdated as INT) as Updates  
FROM
  (describe history mypatronage_test)
where
  operation not in ("CREATE TABLE", "OPTIMIZE")
order by
  1 desc;

In [0]:
%fs 
head 'dbfs:/mnt/ci-carma/landing/caregiverevent-4ea51390-55a9-11ef-a8b3-02f025e39701.csv'

In [0]:
%sql
SHOW PARTITIONS delta.`dbfs:/user/hive/warehouse/mypatronage_test`

In [0]:
%sql
with person(name, last_name, street, city, zip) as
  (values('Bhavesh', 'Patel', '109 Austin St', 'TF', 07712) )
select name, last_name,
  struct(* EXCEPT (name, last_name)) AS address, --removes fields name and last_name
  struct(* ) AS Full_Info
from person

In [0]:
%sql
with person(name, last_name, address) as
  (values('Bhavesh', 'Patel', 
          named_struct('street', '109 Austin St', 
                        'city','TF', 
                        'zip', 07712) ))
select *
-- * except (address), address.* except(street)
  from person

In [0]:
%sql
with person(name, last_name, street, city, zip) as
  (values('Bhavesh', 'Patel', '109 Austin St', 'TF', 07712) )
select concat_ws('',*), 
      concat_ws(',',*)
from person

In [0]:

dbutils.widgets.dropdown("time_period", "Weekly", ["Weekly", "Monthly"])
time_period = dbutils.widgets.get("time_period")
print(time_period)
today = date.today()
if time_period == 'Weekly':
    start_date = today - timedelta(days=today.weekday(), weeks=1) - timedelta(days=1)
    end_date = start_date + timedelta(days=6)
else:
    first = today.replace(day=1)
    end_date = first - timedelta(days=1)
    start_date = first - timedelta(days=end_date.day)

print(start_date,  end_date)



In [0]:
%python
spark.sql("select * from {df}", df=df).display()

In [0]:
%python

def get_summary(input_df):
    """
    Get summary statistics for a dataframe.
    Input: Spark dataframe
    Ouptput: Spark dataframe with summary statistics: Total Records, Nulls, Distincts, Dups, Non-Nulls, Max, Min and % of Nulls
    """
    # Total Records
    total_records = input_df.count()

    # Null Counts
    null_counts = input_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in input_df.columns])
    null_counts_dict = null_counts.first().asDict()

    # Total Non Null Counts
    counts = input_df.select([count(col(c)).alias(c) for c in input_df.columns])
    counts_dict = counts.first().asDict()

    # Count distinct
    distinct_counts_per_column = {}
    for column in input_df.columns:
        distinct_counts = input_df.select(countDistinct(col(column)).alias(column)).first()[column]
        distinct_counts_per_column[column] = distinct_counts

    total_dups_dict = {column: counts_dict[column] - distinct_counts_per_column[column] for column in input_df.columns}

    # max for each column
    max_value = {col: input_df.agg(max(col)).collect()[0][0] for col in input_df.columns}

    # min for each column
    min_value = {col: input_df.agg(min(col)).collect()[0][0] for col in input_df.columns}

    # % of nulls
    null_percent = {column: "{:.2f}".format((null_counts_dict[column] / total_records) * 100) for column in input_df.columns }
    # Create a summary DataFrame
    summary_data = [{"CountType": "Total Records", **{c: total_records for c in input_df.columns}},
                    {"CountType": "Total Nulls", **null_counts_dict},
                    {"CountType": "Null Percentage", **null_percent},
                    {"CountType": "Distinct Values", **distinct_counts_per_column},
                    {"CountType": "Total Dups", **total_dups_dict},
                    {"CountType": "Total Non-Nulls", **counts_dict},
                    {"CountType": "Max Value", **max_value},
                    {"CountType": "Min Value", **min_value},
                    ]


    summary_df = spark.createDataFrame(summary_data)

    # Reorder columns
    summary_df = summary_df.select("CountType", *input_df.columns)

    return summary_df

# Example usage
# df = spark.table('test_correlations')
# display(get_summary(df))

In [0]:
def get_any_id(df, input_value):
    conditions = [df[col].isin(input_value) for col in df.columns]
    df_filtered = df.filter(reduce(lambda a, b: a | b, conditions))

    return df_filtered

# Example usage
# df = spark.table("test_correlations")
# df = df.drop("Last_updated_date") # remove all non numerical columns
# input_value = [1000669101, 24116350, 1082816891]
# result_df = get_any_id(df, input_value)
# display(result_df)

In [0]:
%python
spark.sql("select MVIPersonICN, Inst.* FROM {df} lateral view explode(InstitutionIDS) as Inst where Inst.InstitutionID = '200DOD' ",  df=json_format).display()

In [0]:
%sql
select * from  
read_files('/mnt/ci-vadir-shared/CPIDODIEX_20241005_spool.csv', format=>'csv', header=>true) 
where PTCPNT_ID in (40336746, 3453321, 45264911, 29646885, 57919878, 52213693, 43597650, 47771523, 37241976, 51983424, 7839780)

In [0]:
%sql
select * from mypatronage_test where Status_Begin_Date is null and Batch_Cd = 'SCD'

In [0]:
%sql
select * from  
read_files('/mnt/ci-vadir-shared/CPIDODIEX_20241005_spool.csv', format=>'csv', header=>true) 
where PTCPNT_ID in (SELECT participant_id
FROM delta.`/mnt/Patronage/SCD_Staging`
WHERE Status_Begin_Date IS NULL AND SDP_Event_Created_Timestamp = "2024-10-05T00:00:00.000+00:00" )

In [0]:
%sql
SELECT
  version,
  operation,
  cast(operationMetrics.numTargetRowsInserted as INT) as Inserted,
  cast(operationMetrics.numTargetRowsUpdated as INT) as Updates  
FROM
  (describe history mypatronage_test)
where
  operation not in ("CREATE TABLE", "OPTIMIZE")
order by
  1 desc;

In [0]:
file_list_schema = StructType(
    [
        StructField("path", StringType()),
        StructField("name", StringType()),
        StructField("size", StringType()),
        StructField("modificationTime", StringType()),
    ]
)
def all_file_list(path, cg_unix_start_time):
    """
    Creates a List of files that are at any depth inside a directory
    Parameter: Path of a directory
    Returns: Flat List with file names and it's metadata
    """
    items = dbutils.fs.ls(path)
    files = []

    for item in items:
        if item.isDir():
            # If it's a directory, recursively list its contents
            files += all_file_list(item.path, cg_unix_start_time)
        elif (
            (
                item.name.startswith("caregiverevent")
                and item.name.endswith(".csv")
                and item.modificationTime > cg_unix_start_time
            )
            or (
                (item.name.startswith("CPIDODIEX_") and item.name.endswith(".csv"))
                and "NEW" not in item.name
            )
            or (item.name.startswith("WRTS") and item.name.endswith(".txt"))
            or (item.path.startswith("dbfs:/mnt/vac20sdpasa201vba/ci-vba-edw-2/") and item.name.endswith(".parquet"))
        ):
            files.append((item.path, item.name, item.size, item.modificationTime))

    return files

def collect_data_source():
    """
    This is a helper function that will create a pyspark dataframe with all the files required to process
    Returns: Pyspark dataframe with files that are ready to process from all the
    """
    cg_start_time = datetime(
        2024, 12, 18, 23, 59, 59
    )  # this is the new datetime(2024, 12, 18, 23, 59, 59) for Caregivers data
    cg_unix_start_time = int(time.mktime(cg_start_time.timetuple())) * 1000

    others_start_time = datetime(2023, 11, 1, 0, 0, 0) # Start datetime for SCD and PAI data
    others_unix_start_time = int(time.mktime(others_start_time.timetuple())) * 1000

    # print(others_unix_start_time)

    raw_file_folders = [
        "/mnt/ci-carma/landing/",
        "/mnt/ci-vadir-shared/",
        "/mnt/ci-patronage/pai_landing/",
        "/mnt/vac20sdpasa201vba/ci-vba-edw-2/DeltaTables/DW_ADHOC_RECURR.DOD_PATRONAGE_SCD_PT/",
    ]
    master_file_list = []

    for files in raw_file_folders:
        master_file_list.append(all_file_list(files, cg_unix_start_time))
    flattened_file_list = [item for sublist in master_file_list for item in sublist]
    # print(flattened_file_list)

    file_list_df = spark.createDataFrame(flattened_file_list, file_list_schema)
    # display(file_list_df)

    filtered_file_list_df = (
        file_list_df.filter(file_list_df["modificationTime"] > others_unix_start_time)
        .withColumn("dateTime", to_timestamp(col("modificationTime") / 1000))
        .orderBy(col("modificationTime"))
    )

    no_of_files = spark.sql(
        "SELECT COALESCE(COUNT(DISTINCT filename), 0) AS count from mypatronage_test"
    ).collect()[0][0]
    # print(f"Total files processed till last run: {no_of_files}")

    # display(filtered_file_list_df)
    query = f"""SELECT COALESCE(MAX(SDP_Event_Created_Timestamp), TIMESTAMP('{others_start_time}')) AS max_date from mypatronage_test """
    # print(query)
    if no_of_files == 1:
        max_processed_date = others_start_time
    else:
        max_processed_date = spark.sql(query).collect()[0][0]

    files_to_process_now = filtered_file_list_df.filter(
        filtered_file_list_df["dateTime"] > max_processed_date
    ).orderBy(col("modificationTime"))

    now = datetime.now()
    today_start_time = datetime(now.year, now.month, now.day)
    yesterday_end_time = today_start_time - timedelta(
        hours=4
    )  # To match the time of file creation date in blob storage
    yesterday_end_time_ts = int(yesterday_end_time.timestamp() * 1000)

    # files_to_process_now = files_to_process_now.filter(
    #     files_to_process_now["modificationTime"] <= yesterday_end_time_ts
    # )
    print(
        f"Upserting all files landed between date: {max_processed_date} and date: {yesterday_end_time} "
    )
    # display(files_to_process_now)
    if files_to_process_now.count() != 0:
        return files_to_process_now
    else:
        # dbutils.notebook.exit("Notebook exited because there are no files to upsert.")
        pass


In [0]:
display(dbutils.fs.ls("/mnt/vac20sdpasa201vba/"))

In [0]:
files_to_process = collect_data_source()
display(files_to_process)

In [0]:
from ftplib import FTP
ip_add = '140.20.107.4'
username = 'patelbn'
password = 'w&zuyHqy_XX5zXe4'

In [0]:
ftp = FTP(ip_add)
ftp.login(username, password)

In [0]:
%sql
SELECT DISTINCT data_date
FROM DELTA.`/mnt/vac20sdpasa201vba/ci-vba-edw-2/DeltaTables/DW_ADHOC_RECURR.DOD_PATRONAGE_SCD_PT/`
ORDER BY DATA_DATE DESC
-- 2024-10-08 00:00:00.0000000

In [0]:
%sql
SELECT count(distinct PT35_PRMLGN_DT)
FROM DELTA.`/mnt/vac20sdpasa201vba/ci-vba-edw-2/DeltaTables/DW_ADHOC_RECURR.DOD_PATRONAGE_SCD_PT/`
-- ORDER BY PT35_PRMLGN_DT DESC
-- LIMIT 1

In [0]:
%sql
SELECT *
FROM delta.`/mnt/Patronage/identity_correlations`
WHERE participant_id = 63371531

In [0]:
%sql
SELECT *
FROM delta.`/mnt/Patronage/duplicate_identity_correlations`
-- WHERE TreatingFacilityPersonIdentifier = 63371531
WHERE MVIPersonICN = 1008568848

In [0]:
%sql
CREATE TABLE Employee (EmpNo INT, ProjId INT, Dept INT);
INSERT INTO Employee VALUES (1,1,10);
INSERT INTO Employee VALUES (1,2,10);
INSERT INTO Employee VALUES (2,1,20);
INSERT INTO Employee VALUES (2,2,20);
INSERT INTO Employee VALUES (3,1,30);
SELECT EmpNo, COLLECT_LIST(ProjId) AS ProjId, Dept FROM Employee GROUP BY EmpNo, Dept;

In [0]:
%sql
SELECT EmpNo, collect_list(struct(Dept, ProjId)) AS ProjId FROM Employee GROUP BY EmpNo, Dept; 

In [0]:
%sql
SELECT PT_Indicator, count(1)
FROM mypatronage_test
WHERE Batch_CD ='SCD' and RecordStatus IS TRUE
GROUP BY 1

In [0]:
%sql
CREATE TEMP VIEW temp_pat as SELECT * FROM delta.`/mnt/Patronage/SCD_Staging` VERSION AS OF 128

In [0]:
%sql
SELECT d.PTCPNT_VET_ID as d_Participant_ID, d.PT_35_FLAG as d_PT, p.participant_id as pat_id, p.PT_Indicator pat_PT  FROM DELTA.`/mnt/vac20sdpasa201vba/ci-vba-edw-2/DeltaTables/DW_ADHOC_RECURR.DOD_PATRONAGE_SCD_PT/` d
JOIN temp_pat p ON d.PTCPNT_VET_ID = p.participant_id
WHERE d.PT_35_FLAG != p.PT_Indicator

In [0]:
%sql
SELECT count(*)
-- d.PTCPNT_VET_ID as d_Participant_ID, d.PT_35_FLAG as d_PT, p.participant_id as pat_id, p.PT_Indicator pat_PT 
FROM DELTA.`/mnt/vac20sdpasa201vba/ci-vba-edw-2/DeltaTables/DW_ADHOC_RECURR.DOD_PATRONAGE_SCD_PT/` d
JOIN temp_pat p ON d.PTCPNT_VET_ID = p.participant_id
WHERE d.PT_35_FLAG != p.PT_Indicator

In [0]:
%sql
SELECT *
FROM mypatronage_test
WHERE ICN NOT IN (SELECT ICN FROM delta.`/mnt/Patronage/SCD_Staging`)
AND BATCH_CD = 'SCD'

In [0]:
%sql
SELECT * FROM delta.`/mnt/Patronage/SCD_Staging`
WHERE ICN NOT IN (SELECT DISTINCT ICN 
FROM mypatronage_test WHERE
BATCH_CD = 'SCD')

In [0]:
%sql
SELECT * FROM delta.`/mnt/Patronage/SCD_Staging`
WHERE ICN = 1079152916

In [0]:
icn_relationship = (spark.read.format("delta")
                    .load("/mnt/Patronage/identity_correlations")
                    .withColumnRenamed('MVIPersonICN', 'ICN')).persist()

In [0]:
%sql
SELECT * from delta.`/mnt/Patronage/duplicate_identity_correlations`
WHERE TreatingFacilityPersonIdentifier = 64246516

In [0]:
%sql
SELECT * from delta.`/mnt/Patronage/identity_correlations`
WHERE participant_id = 64246516

In [0]:
%sql
SELECT
  version,
  operation,
  cast(operationMetrics.numTargetRowsInserted as INT) as Inserted,
  cast(operationMetrics.numTargetRowsUpdated as INT) as Updates  
FROM
  (describe history delta.`/mnt/Patronage/SCD_Staging`)
where
  operation not in ("CREATE TABLE", "OPTIMIZE")
order by
  1 desc;

In [0]:
%sql
SELECT EXPLODE(collect_set(SC_Combined_Disability_Percentage))
FROM mypatronage_test WHERE BATCH_CD = 'SCD'

In [0]:
%sql
SELECT *
FROM (
  SELECT SC_Combined_Disability_Percentage, count(*) over() AS TotRec
  FROM mypatronage_test WHERE BATCH_CD = 'SCD'
)
PIVOT (
  count(SC_Combined_Disability_Percentage)
  FOR SC_Combined_Disability_Percentage IN ('010', '020'
  )
)

In [0]:
%sql
SELECT SC_Combined_Disability_Percentage, count(*) over() AS TotRec
  FROM mypatronage_test WHERE BATCH_CD = 'SCD'
  

In [0]:
from ftplib import FTP
ip_add = '140.20.107.4'
username = 'patelbn'
password = 'w&zuyHqy_XX5zXe4'
ftp_directory = '/ftp/data02/pftbat/incoming' 


In [0]:
ftp = FTP()
ftp.connect(host=ip_add, port=21, timeout=60)
ftp.login(user=username, passwd=password)
ftp.set_pasv(True)

In [0]:
%sql
  select EDIPI,
          Batch_CD,
          SC_Combined_Disability_Percentage,
          status_begin_date,
          PT_Indicator,
          Individual_Unemployability,
          Status_Last_Update,
          Status_Termination_Date
          from mypatronage_test111
        where Batch_CD = 'CG'
        and SDP_Event_Created_Timestamp >= DATE('2025-05-16')
        and SDP_Event_Created_Timestamp <= DATE('2025-05-20')
        and EDIPI is not null
        and Applicant_Type = 'Primary Caregiver'
        and (Status_Termination_Date is NULL OR Status_Termination_Date >= curdate()
            OR Caregiver_Status IN ('Approved', 'Pending Revocation/Discharge'))
        and EDIPI is not null
        and sentToDoD is false

In [0]:
%sql
select count(*)
            -- edipi,
            -- Batch_CD,
            -- SC_Combined_Disability_Percentage,
            -- Status_Begin_Date,
            -- PT_Indicator,
            -- Individual_Unemployability,
            -- Status_Last_Update,
            -- Status_Termination_Date
        from mypatronage_test111
        where Batch_CD = 'SCD'
        and SDP_Event_Created_Timestamp >= DATE('2025-05-16')
        and SDP_Event_Created_Timestamp <= DATE('2025-05-20')
        and edipi is not null
        and sentToDoD is false

In [0]:
20196 + 155

In [0]:
%sql
  select rpad(coalesce(EDIPI,''),10,' ' ) ||
            rpad(coalesce(Batch_CD,''), 3,' ') ||
            rpad(coalesce(SC_Combined_Disability_Percentage,''),3,' ') ||
            rpad(coalesce(date_format(status_begin_date, 'yyyyMMdd'),''),8,' ') ||
            rpad(coalesce(PT_Indicator,''),1,' ') ||
            rpad(coalesce(Individual_Unemployability,''),1,' ') ||  
            rpad(coalesce(date_format(Status_Last_Update, 'yyyyMMdd'),''),8,' ') ||
            rpad(coalesce(date_format(Status_Termination_Date, 'yyyyMMdd'),''),8,' ') as CG
        from mypatronage_test111
        where Batch_CD = 'CG'
        and SDP_Event_Created_Timestamp >= DATE('2025-05-16')
        and SDP_Event_Created_Timestamp <= DATE('2025-05-20')
        and EDIPI is not null
        and Applicant_Type = 'Primary Caregiver'
        and (Status_Termination_Date is NULL OR Status_Termination_Date >= curdate()
            OR Caregiver_Status IN ('Approved', 'Pending Revocation/Discharge'))
        and EDIPI is not null

In [0]:
%sql
  select rpad(coalesce(EDIPI,''),10,' ' ) ||
            rpad(coalesce(Batch_CD,''), 3,' ') ||
            rpad(coalesce(SC_Combined_Disability_Percentage,''),3,' ') ||
            rpad(coalesce(date_format(status_begin_date, 'yyyyMMdd'),''),8,' ') ||
            rpad(coalesce(PT_Indicator,''),1,' ') ||
            rpad(coalesce(Individual_Unemployability,''),1,' ') ||  
            rpad(coalesce(date_format(Status_Last_Update, 'yyyyMMdd'),''),8,' ') ||
            rpad(coalesce(date_format(Status_Termination_Date, 'yyyyMMdd'),''),8,' ') as CG
        from mypatronage_test111
        where Batch_CD = 'CG'
        and SDP_Event_Created_Timestamp >= DATE('2025-05-16')
        and SDP_Event_Created_Timestamp <= DATE('2025-05-20')
        and EDIPI is not null
        and Applicant_Type = 'Primary Caregiver'
        and (Status_Termination_Date is NULL OR Status_Termination_Date >= curdate()
            OR Caregiver_Status IN ('Approved', 'Pending Revocation/Discharge'))
        and EDIPI is not null
        
        union all
        
        select rpad(coalesce(edipi,''),10,' ' ) ||
            rpad(coalesce(Batch_CD,''), 3,' ') ||
            rpad(coalesce(SC_Combined_Disability_Percentage,''),3,' ') ||
            rpad(coalesce(Status_Begin_Date,''),8,' ') ||
            rpad(coalesce(PT_Indicator,''),1,' ') ||
            rpad(coalesce(Individual_Unemployability,''),1,' ') ||
            rpad(coalesce(Status_Last_Update,''),8,' ') ||
            rpad(coalesce(Status_Termination_Date,''),8,' ') as CG
        from scd_data
        where SDP_Event_Created_Timestamp >= DATE('2025-05-16')
        and SDP_Event_Created_Timestamp <= DATE('2025-05-20')
        and edipi is not null